In [34]:
import os

# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")

In [37]:
from glob import glob

text_lines = []

for file_path in glob("mfd.md", recursive=True):
    with open(file_path, "r") as file:
        file_text = file.read()

    line = file_text.split(" ")
    text_lines += line

In [38]:
len(text_lines)

441

In [4]:
from openai import OpenAI

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

In [5]:
from pymilvus import model as milvus_model

embedding_model = milvus_model.DefaultEmbeddingFunction()

/opt/conda/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./milvus_mfd.db")

collection_name = "mfd_rag_collection"

In [7]:
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

In [8]:
test_embedding = embedding_model.encode_queries(["第四百零七条"])[0]
embedding_dim = len(test_embedding)
print(embedding_dim)
print(test_embedding[:10])

768
[-0.00993222  0.04682298 -0.03675228 -0.08068266 -0.02617284  0.0037349
  0.00990155 -0.08762875 -0.01553682  0.05377049]


In [9]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # 内积距离
    consistency_level="Strong",  # 支持的值为 (`"Strong"`, `"Session"`, `"Bounded"`, `"Eventually"`)。更多详情请参见 https://milvus.io/docs/consistency.md#Consistency-Level。
)

In [10]:
from tqdm import tqdm

data = []

doc_embeddings = embedding_model.encode_documents(text_lines)

for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i], "text": line})

milvus_client.insert(collection_name=collection_name, data=data)

Creating embeddings: 100%|██████████| 441/441 [00:00<00:00, 601916.06it/s]


{'insert_count': 441, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 

In [27]:
question = "国家、集体、私人的物权和其他权利人的物权受法律平等保护，任何组织或者个人不得侵犯是属于那一条的内容?"

In [28]:
search_res = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode_queries(
        [question]
    ),  # 将问题转换为嵌入向量
    limit=5,  # 返回前3个结果
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)

In [29]:
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))

[
    [
        "\u6cd5\u5f8b\u53e6\u6709\u89c4\u5b9a\u7684\uff0c\u4f9d\u7167\u5176\u89c4\u5b9a\u3002",
        0.906079888343811
    ],
    [
        "\u56e0\u5408\u6cd5\u5efa\u9020\u3001\u62c6\u9664\u623f\u5c4b\u7b49\u4e8b\u5b9e\u884c\u4e3a\u8bbe\u7acb\u6216\u8005\u6d88\u706d\u7269\u6743\u7684\uff0c\u81ea\u4e8b\u5b9e\u884c\u4e3a\u5b8c\u6210\u65f6\u53d1\u751f\u6548\u529b\u3002\n\n**\u7b2c\u4e8c\u767e\u4e09\u5341\u4e8c\u6761**",
        0.809916615486145
    ],
    [
        "\u56e0\u7ee7\u627f\u53d6\u5f97\u7269\u6743\u7684\uff0c\u81ea\u7ee7\u627f\u5f00\u59cb\u65f6\u53d1\u751f\u6548\u529b\u3002\n\n**\u7b2c\u4e8c\u767e\u4e09\u5341\u4e00\u6761**",
        0.809916615486145
    ],
    [
        "\u5229\u5bb3\u5173\u7cfb\u4eba\u53ef\u4ee5\u7533\u8bf7\u67e5\u8be2\u4e0d\u52a8\u4ea7\u767b\u8bb0\u8d44\u6599\u3002\u7533\u8bf7\u67e5\u8be2\u7684\uff0c\u767b\u8bb0\u673a\u6784\u5e94\u5f53\u63d0\u4f9b\u3002\n\n**\u7b2c\u4e8c\u767e\u4e8c\u5341\u6761**",
        0.809916615486145
    ],
    [
        

In [30]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)

In [31]:
context

'法律另有规定的，依照其规定。\n因合法建造、拆除房屋等事实行为设立或者消灭物权的，自事实行为完成时发生效力。\n\n**第二百三十二条**\n因继承取得物权的，自继承开始时发生效力。\n\n**第二百三十一条**\n利害关系人可以申请查询不动产登记资料。申请查询的，登记机构应当提供。\n\n**第二百二十条**\n权利人、利害关系人可以申请查询、复制不动产登记资料，登记机构应当提供。\n\n**第二百一十九条**'

In [32]:
SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。
<context>
{context}
</context>
<question>
{question}
</question>
"""

In [33]:
USER_PROMPT

'\n请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。\n<context>\n法律另有规定的，依照其规定。\n因合法建造、拆除房屋等事实行为设立或者消灭物权的，自事实行为完成时发生效力。\n\n**第二百三十二条**\n因继承取得物权的，自继承开始时发生效力。\n\n**第二百三十一条**\n利害关系人可以申请查询不动产登记资料。申请查询的，登记机构应当提供。\n\n**第二百二十条**\n权利人、利害关系人可以申请查询、复制不动产登记资料，登记机构应当提供。\n\n**第二百一十九条**\n</context>\n<question>\n国家、集体、私人的物权和其他权利人的物权受法律平等保护，任何组织或者个人不得侵犯是属于那一条的内容?\n</question>\n'

In [113]:
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)

根据提供的上下文，第二百零四条的内容是："用益物权人不得转让、出租、担保其用益物权"。

这个答案是从给定的法律条款片段中直接提取的，这些条款看起来像是中国《民法典》物权编的相关规定。虽然上下文没有明确标注条款编号，但根据问题的询问方式可以推断这是在询问某个特定条款（第二百零四条）的内容。
